In [1]:
import os, shutil, time, sys, pdb
import numpy as np, pandas as pd, time, os, shutil, sys
from os.path import join

from torchtext.data import Field, TabularDataset, BucketIterator
import torch, torchtext
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torchtext.vocab import Vectors, GloVe
from torchtext.datasets import IMDB
from sklearn.model_selection import train_test_split

In [2]:
from torch_trainer.engine import Model
from torch_trainer.metrics import accuracy
from torch_trainer.callbacks import CyclicalLearningRate, ModelCheckpoint
from torch_trainer.callbacks.logger import NeptuneLogger
from torch_trainer.utils.lr_finder import LRFinder
from torch_trainer.optimizers import RAdam, Lookahead

In [3]:
TEXT = Field(lower=True, include_lengths=True, batch_first=True, fix_length=90)
LABEL = Field(sequential=False)

# make splits for data
train, test = IMDB.splits(TEXT, LABEL)

# build the vocabulary
MAX_VOCAB_SIZE = 25000
TEXT.build_vocab(train, vectors=GloVe(name='6B', dim=100), max_size = MAX_VOCAB_SIZE, unk_init = torch.Tensor.normal_)
LABEL.build_vocab(train)
vocab_size = len(TEXT.vocab)

# make iterator for splits
train_iter, test_iter = BucketIterator.splits((train, test), batch_size=256, device=torch.device('cpu'))

In [4]:
embedding_size = 100
hidden_size = 256
output_size = 2

class GRUClassifier(Model):
    
    def __init__(self):
        super(GRUClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.embedding.weight = nn.Parameter(TEXT.vocab.vectors, requires_grad=True)
        
        self.gru = nn.GRU(embedding_size, hidden_size, batch_first=False, bidirectional=True, dropout=0.4, num_layers=2)
        self.dropout = nn.Dropout(0.5)
        self.label = nn.Linear(hidden_size*4, output_size)    
        
    def forward(self, z):
        z = self.dropout(self.embedding(z).transpose(0,1))
        out, h_state = self.gru(z)
        concat_hidden = torch.flatten(h_state.transpose(0,1), start_dim=1, end_dim=2)
#         logits = self.label(concat_hidden)
        logits = self.label(self.dropout(concat_hidden))
        return logits
    
    @staticmethod
    def parse_input(batch):
        return batch.text[0], batch.label-1

class RNN(Model):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.embedding.weight = nn.Parameter(TEXT.vocab.vectors, requires_grad=True)
        self.rnn = nn.LSTM(embedding_size, 
                           hidden_size, 
                           num_layers=2, 
                           bidirectional=True, 
                           dropout=0.5)
        
        self.fc = nn.Linear(hidden_size * 2, output_size)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, text):
        embedded = self.dropout(self.embedding(text).transpose(0,1))
        packed_output, (hidden, cell) = self.rnn(embedded)
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        return self.fc(hidden)
    
    @staticmethod
    def parse_input(batch):
        return batch.text[0], batch.label-1

In [5]:
# model = GRUClassifier()
model = RNN()
opt = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()
model.compile(optimizer=opt, metrics=[accuracy], loss=criterion)

In [6]:
cpkt = ModelCheckpoint('models/test.pth', monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True)
cyclic_lr = CyclicalLearningRate(base_lr=2e-3, max_lr=5e-2, epochs_per_cycle=3, auto_find_lr=False)

In [ ]:
hist = model.fit(train_iter, epochs=5, val_dataloader=test_iter)


Epoch 1/5
96/98 [============================>.] - ETA: 0s - loss: 0.6558 - accuracy: 67.5781